In [ ]:
!pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.9 MB/s 
     |████████████████████████████████| 5.3 MB 72.5 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 212 kB 72.6 MB/s 
     |████████████████████████████████| 163 kB 71.2 MB/s 
     |████████████████████████████████| 115 kB 79.3 MB/s 
     |████████████████████████████████| 127 kB 84.0 MB/s 
     |████████████████████████████████| 7.6 MB 74.3 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=0bab3a67234fa1f4211cb758505762b93b84d2072ed8acb73181c9dc77d5559b
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
!huggingface-cli login



        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

# **Fine-tuning a model on a token classification task**

In [ ]:
import transformers

print(transformers.__version__)

4.23.1


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "alexaapo/greek_legal_bert_v2"
batch_size = 16

In [ ]:
#connect to G-Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **Preprocessing the data**

Import Data

In [ ]:
import ast

with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/train_texts.txt') as f:
    train_texts = [ast.literal_eval(line) for line in f]

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/train_tags.txt') as f:
    train_tags = [ast.literal_eval(line) for line in f]

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/val_texts.txt') as f:
    val_texts = [ast.literal_eval(line) for line in f]

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/val_tags.txt') as f:
    val_tags = [ast.literal_eval(line) for line in f]

Dictionary of Tags to IDs and vise-versa

In [ ]:
import json

f = open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/tag2id.txt', 'r')
tag2id = json.loads(f.read())

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/id2tag.txt','r') as f:
   id2tag = ast.literal_eval(f.read())

Unique Tags

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/NER/splits/unique_tags.txt','r') as f:
   unique_tags = ast.literal_eval(f.read())

Ready for the Process

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.0 [00:00<?, ?B/s]

Since our inputs have already been split into words, we pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Encode our Dataset

In [ ]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(val_encodings, val_labels)

In [ ]:
print(train_dataset[1]['input_ids'],train_dataset[1]['labels'])

tensor([    2,    12,  1356,    13,   127,  2768,   426, 10668,  5255,   412,
         3554,  6547,  6766,   414,   426,  3120, 10633,   426,  1625,  1012,
          453,   434,  2029,   412,  1555,   426,   125,    18,   125,    18,
         4650,  3042,  2184,   414, 12191,   842,   421,   132,    18,   125,
           18,   121,    18,    17,   125,    18,   125,    18,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
len(train_dataset[1]['input_ids'])


410

## **Fine-tuning the model**

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(unique_tags), id2label= id2tag, label2id= tag2id)

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

Some weights of the model checkpoint at alexaapo/greek_legal_bert_v2 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "steps", 
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps=250,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    metric_for_best_model='eval_f1',
    load_best_model_at_end = True,)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
from datasets import load_metric
metric = load_metric("seqeval")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


This metric takes list of labels for the predictions and references:

In [ ]:
labels = [id2tag[i] for i in train_labels[0] if i!= -100]
metric.compute(predictions=[labels], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 5},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,           # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Cloning https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/429M [00:00<?, ?B/s]

Download file runs/Sep15_15-50-02_a66885e3bcb4/1663257189.6141803/events.out.tfevents.1663257189.a66885e3bcb4.…

Download file runs/Sep15_15-50-02_a66885e3bcb4/events.out.tfevents.1663257189.a66885e3bcb4.77.0:  15%|#4      …

Download file runs/Sep08_09-20-50_cf69a67d4524/1662628880.1003811/events.out.tfevents.1662628880.cf69a67d4524.…

...

Clean file pytorch_model.bin:   0%|          | 1.00k/429M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6197
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3880


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
250,No log,0.083950,0.785917,0.853859,0.818481,0.973723
500,0.112700,0.078349,0.809200,0.856947,0.832389,0.975903
750,0.112700,0.074275,0.828398,0.844597,0.836419,0.976636
1000,0.053800,0.081583,0.824342,0.859691,0.841646,0.977369
1250,0.053800,0.090002,0.842422,0.863808,0.852981,0.977573
1500,0.034600,0.088956,0.840094,0.859691,0.849780,0.977023
1750,0.034600,0.096408,0.834168,0.857633,0.845737,0.976758
2000,0.023300,0.109372,0.833609,0.864494,0.848771,0.976758
2250,0.023300,0.111025,0.845606,0.854889,0.850222,0.977716
2500,0.016100,0.122394,0.840825,0.853516,0.847123,0.976860


***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
Saving model checkpoint to greek_legal_bert_v2-finetuned-ner/checkpoint-500
Configuration saved in greek_legal_bert_v2-finetuned-ner/checkpoint-500/config.json
Model weights saved in greek_legal_bert_v2-finetuned-ner/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
Saving model checkpoint to greek_legal_bert_v2-finetuned-ner/checkpoint-1000
Configuration saved in greek_legal_bert_v2-finetuned-ner/checkpoint-1000/config.json
Model weights saved in greek_legal_bert_v2-finetuned-ner/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16
Saving model checkpoint to greek_legal_bert_v2-finetuned-ner/checkpoint

TrainOutput(global_step=3880, training_loss=0.03424658240731229, metrics={'train_runtime': 1420.7809, 'train_samples_per_second': 43.617, 'train_steps_per_second': 2.731, 'total_flos': 1.29668156805834e+16, 'train_loss': 0.03424658240731229, 'epoch': 10.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1550
  Batch size = 16


{'eval_loss': 0.0889558345079422,
 'eval_precision': 0.8400938652363392,
 'eval_recall': 0.8596912521440824,
 'eval_f1': 0.8497795863004407,
 'eval_accuracy': 0.9770232008636669,
 'eval_runtime': 10.8732,
 'eval_samples_per_second': 142.553,
 'eval_steps_per_second': 8.921,
 'epoch': 10.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 1550
  Batch size = 16


{'ORG': {'precision': 0.8400938652363392,
  'recall': 0.8596912521440824,
  'f1': 0.8497795863004407,
  'number': 2915},
 'overall_precision': 0.8400938652363392,
 'overall_recall': 0.8596912521440824,
 'overall_f1': 0.8497795863004407,
 'overall_accuracy': 0.9770232008636669}

## Push to Hub Directly


In [ ]:
trainer.push_to_hub()

Saving model checkpoint to greek_legal_bert_v2-finetuned-ner
Configuration saved in greek_legal_bert_v2-finetuned-ner/config.json
Model weights saved in greek_legal_bert_v2-finetuned-ner/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/429M [00:00<?, ?B/s]

Upload file runs/Oct19_08-40-32_cd2669a0e78e/events.out.tfevents.1666170484.cd2669a0e78e.94.2: 100%|##########…

Upload file runs/Oct19_08-40-32_cd2669a0e78e/events.out.tfevents.1666169052.cd2669a0e78e.94.0:  28%|##7       …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner
   13370ab..c942e35  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner
   13370ab..c942e35  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.8400938652363392}, {'name': 'Recall', 'type': 'recall', 'value': 0.8596912521440824}, {'name': 'F1', 'type': 'f1', 'value': 0.8497795863004407}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9770232008636669}]}
To https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner
   c942e35..0029c46  main -> main

   c942e35..0029c46  main -> main



'https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner/commit/c942e35ee6ee27019fe147e0e5880784eada9e35'

In [ ]:
tokenizer.push_to_hub('greek_legal_bert_v2-finetuned-ner')

tokenizer config file saved in greek_legal_bert_v2-finetuned-ner/tokenizer_config.json
Special tokens file saved in greek_legal_bert_v2-finetuned-ner/special_tokens_map.json
Uploading the following files to amichailidis/greek_legal_bert_v2-finetuned-ner: tokenizer.json,tokenizer_config.json,special_tokens_map.json,vocab.txt


CommitInfo(commit_url='https://huggingface.co/amichailidis/greek_legal_bert_v2-finetuned-ner/commit/123d686f1ae878c986a517440ca153425a9a7fe9', commit_message='Upload tokenizer', commit_description='', oid='123d686f1ae878c986a517440ca153425a9a7fe9', pr_url=None, pr_revision=None, pr_num=None)

## Save Model Locally

In [ ]:
model.save_pretrained("gdrive/MyDrive/path/to/model2")


Configuration saved in gdrive/MyDrive/path/to/model2/config.json
Model weights saved in gdrive/MyDrive/path/to/model2/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained("gdrive/MyDrive/path/to/model2")


tokenizer config file saved in gdrive/MyDrive/path/to/model2/tokenizer_config.json
Special tokens file saved in gdrive/MyDrive/path/to/model2/special_tokens_map.json


('gdrive/MyDrive/path/to/model2/tokenizer_config.json',
 'gdrive/MyDrive/path/to/model2/special_tokens_map.json',
 'gdrive/MyDrive/path/to/model2/vocab.txt',
 'gdrive/MyDrive/path/to/model2/added_tokens.json',
 'gdrive/MyDrive/path/to/model2/tokenizer.json')

## Perform tests

In [ ]:
model_test = AutoModelForTokenClassification.from_pretrained('/content/gdrive/MyDrive/path/to/model2')    
tokenizer_test = AutoTokenizer.from_pretrained('/content/gdrive/MyDrive/path/to/model2')

loading configuration file /content/gdrive/MyDrive/path/to/model2/config.json
Model config BertConfig {
  "_name_or_path": "/content/gdrive/MyDrive/path/to/model2",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "I-ORG",
    "2": "B-ORG"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-ORG": 2,
    "I-ORG": 1,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 35000
}

loading weights file /content/gdrive/MyDrive/path/to/model2/pytorch_model.bin
A

In [ ]:
from transformers import TokenClassificationPipeline

pipe = TokenClassificationPipeline(model=model_test, tokenizer=tokenizer_test)
pipe("(3) Γραφείο Συμβούλου Ακεραιότητας (ΓΣΑ)")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-ORG',
  'score': 0.99988246,
  'index': 4,
  'word': 'γραφειο',
  'start': 4,
  'end': 11},
 {'entity': 'I-ORG',
  'score': 0.9998987,
  'index': 5,
  'word': 'συμβουλου',
  'start': 12,
  'end': 21},
 {'entity': 'I-ORG',
  'score': 0.9999453,
  'index': 6,
  'word': 'ακεραιοτητας',
  'start': 22,
  'end': 34},
 {'entity': 'B-ORG',
  'score': 0.999566,
  'index': 8,
  'word': 'γ',
  'start': 36,
  'end': 37},
 {'entity': 'I-ORG',
  'score': 0.9928483,
  'index': 9,
  'word': '##σα',
  'start': 37,
  'end': 39}]